In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['col_0', 'col_1', 'col_2', 'col_3'],
                  index=['row_0', 'row_1', 'row_2'])
print(df)

       col_0  col_1  col_2  col_3
row_0      0      1      2      3
row_1      4      5      6      7
row_2      8      9     10     11


In [3]:
a=[1, 1, 1]
for i in range(len(a)):
    print("w")

w
w
w


In [20]:
import glob
import os
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision.transforms import v2
import torchvision.models as models
import torch.nn as nn
import cv2
from torch.utils.data import Dataset
from pathlib import Path
from typing import List, Tuple
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import pandas as pd

date=20240130
time="0943"
filedate=str(date)+"_"+str(time)
print(filedate)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')

Channels=4
IMG_SIZE=224
target_epoch=60

Classes = ["higainashi", "houdo", "kanbotsu", "rokatahoukai"]
ClassNum = len(Classes)

testpath=r"C:\Users\AKIZUKI\JupyterProjects\QGISedited\test_ishikawa"
savepath=r"C:\Users\AKIZUKI\JupyterProjects\AerialPhoto_savedmodel"
forwardpath=r"C:\Users\AKIZUKI\JupyterProjects\AerialPhoto_Forwarded\20240301ishikawa"

#後で画像ファイル名ごとに推測ラベルを付けるためにfilelistをここで作る
pathlist=list(sorted(Path(testpath).glob("*\*")))
filelist=[]
pathlist_binary=list(sorted(Path(testpath+"binary").glob("*\*")))
filelist_binary=[]
for i in range(len(pathlist)):
    filelist.append(str(pathlist[i]).split("\\")[-1])
    filelist_binary.append(str(pathlist_binary[i]).split("\\")[-1])

print(np.array(filelist).shape)
print(np.array(filelist_binary).shape)


'''
PytorchではDataloaderという,膨大なデータセットからでもメモリを圧迫せずに取り出せてforループにも対応するための枠組みがある
データセットをDataloaderが引っ張ってこれるような形式にするためにMyDataset(torch.utils.data.Dataset)というクラスを作れば，
あとはそのメソッドをtorch.utils.data.Datasetが勝手に使用してデータを加工してくれる
__init__, __getitem__, __len__をクラス内で必ず定義しなければならない
Dataloader内のデータはバッチごとにまとめられる
'''

class MyDataset(Dataset):
    def __init__(self, data, data2, transforms, Classes) -> None:
        super().__init__()
        self.transforms = transforms
        self.Classes = Classes
        #globは複数のファイルのパスをまとめて取得する
        #訓練と訓練白黒の二個下のディレクトリから画像を取得
        self.data = data
        self.data2= data2

    # ここで取り出すデータを指定している
    def __getitem__(
            self,
            index: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:

        data = self.data[index]
        data2 = self.data2[index]

        img1 = cv2.imread(str(data))
        img1 = cv2.resize(img1, (IMG_SIZE, IMG_SIZE))
        img2_tmp = cv2.imread(str(data2))

        # グレースケールに変換
        img2_tmp = cv2.cvtColor(img2_tmp,cv2.COLOR_BGR2GRAY)
        # 2値化
        ret,img2 = cv2.threshold(img2_tmp, 0, 255, cv2.THRESH_BINARY_INV)
        img2 = cv2.resize(img2, (IMG_SIZE, IMG_SIZE))

        #img1のテンソルとimg2のテンソルをチャンネル方向(dim0)に結合
        cat_img = torch.cat((TF.to_tensor(img1), TF.to_tensor(img2)), dim=0)

        # データの変形 (transforms)
        transformed_img = self.transforms(cat_img)

        #ラベル貼り：dataというパスを/で区切ってリストにし，クラス名のところをラベルに格納
        #クラス名は文字列なので，self.Classesの要素と比較して一致するところの番号をラベルとする
        label = str(data).split("\\")[-2]
        label = torch.tensor(self.Classes.index(label))

        return transformed_img, label

    # この method がないと DataLoader を呼び出す際にエラーを吐かれる
    def __len__(self) -> int:
        return len(self.data)

#入力データに施す処理
transforms = v2.Compose([
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0,0,0,0], std=[0.2, 0.2, 0.2, 0.2]),
])

testset= MyDataset(data=pathlist, data2=pathlist_binary, transforms=transforms, Classes=Classes)

testloader = DataLoader(dataset=testset,batch_size=len(testset),shuffle=False)
print(len(testloader))

resnet50 = models.resnet50()

resnet50.conv1 = torch.nn.Conv2d(Channels,64,kernel_size = (7,7),stride = (2,2), padding = (3,3), bias = False)
num_ftrs = resnet50.fc.in_features
#modifying final layer
resnet50.fc = nn.Linear(num_ftrs,ClassNum)

#lossfunction&optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

def evaluate(testloader, model, loss_fn, optimizer):
    size_test = len(testloader.dataset)
    test_loss, test_correct = 0, 0
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in testloader:
            X=X.to(device)
            y=y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_correct /= size_test

    print(f'TestLoss: {test_loss:.4f} TestAcc: {test_correct:.4f}')

    #ndarrayにするため、ラベルyと推測結果predをgpuからcpuへ返す
    y=y.to(cpu)
    pred=pred.to(cpu)

    y=np.array(y)

    #predは各クラスの確率になってる（onehotに近い）ので実際のクラス番号に戻す
    pred_class=pred.argmax(1)
    pred_class=np.array(pred_class)


    #テストデータの混同行列を計算し可視化
    #scikitlearnの混同行列はラベルをonehotではなく実際のクラス番号にする必要がある
    #混同行列の見方は行が正解ラベルのクラス列が推定クラス
    print(confusion_matrix(y, pred_class))
    print(" ")
    csvpath=str(Path(forwardpath+"\\"+filedate+"_ep"+str(target_epoch)+".csv"))
    chart=pd.DataFrame(pred_class, columns=['CLASS'], index=filelist)
    chart.to_csv(csvpath)
    print(chart)
    print("CSV saved in ", csvpath)


#モデル構築
modelpath = Path(savepath+"\\"+str(target_epoch)+"\model_weights"+filedate+".pth")
epochmodel = resnet50
epochmodel.load_state_dict(torch.load(modelpath))
#GPUにニューラルネットワークを渡す
epochmodel=epochmodel.to(device)

print("Model in Epoch", target_epoch)
#テストデータで評価
evaluate(testloader, epochmodel, loss_fn, optimizer)

print('Forwarding Complete!!!')

20240130_0943
(70,)
(70,)
1
Model in Epoch 60
TestLoss: 3.9867 TestAcc: 0.3857
[[ 9  0  3  8]
 [ 5  1  1  4]
 [ 9  0  3  8]
 [ 3  1  1 14]]
 
         CLASS
s1.png       0
s10.png      2
s11.png      3
s12.png      0
s13.png      3
...        ...
30.png       3
4.png        2
6.png        3
8.png        3
9.png        3

[70 rows x 1 columns]
CSV saved in  C:\Users\AKIZUKI\JupyterProjects\AerialPhoto_Forwarded\20240301ishikawa\20240130_0943_ep60.csv
Forwarding Complete!!!
